In [ ]:
using CUDA
using CUDAKernels
using DataFrames
using KernelAbstractions
using Printf
using SparseArrays
using Statistics
using Juliana

# Config

In [ ]:
data_dir = "/data/user/bellotti_r/data"

patient_ID = "train_00"

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.9.jar";

In [ ]:
output_dir = "../output/test/full_dose_calculation"
mkpath(output_dir)

dicom_dir = "$(output_dir)/DICOM"
mkpath(dicom_dir)

In [ ]:
dose_resolution = 0.35f0

In [ ]:
gantry_angle = 90f0
couch_angle = 0f0
nozzle_extraction = 20f0;

# Load data

In [ ]:
ct_path, patient_data = Juliana.load_patient_data(data_dir, patient_ID)

ct = patient_data.ct
target_name, target_dose = Juliana.coldest_target(patient_data.prescriptions)
target = patient_data.structures[target_name];

In [ ]:
# ct, waterblock = Juliana.build_water_slab();

# target = waterblock

In [ ]:
tmp = mean(target.points, dims=1)
iso_center = Dict(
    "x" => tmp[1],
    "y" => tmp[2],
    "z" => tmp[3],
)

In [ ]:
depth_dose_curves, sigma_mcs_curves, phase_space_no_preabsorber, phase_space_with_preabsorber = Juliana.load_machine_parameters(fiona_standalone_bin_path, nozzle_extraction);
ranges = [Juliana.get_range(depth_dose_curves, E) for E in depth_dose_curves.energies];

# Create the plan & calculate the dose using Fiona standalone

In [ ]:
optimisation_mask, optimisation_points, optimisation_indices = Juliana.get_optimisation_points_from_prescription(
    ct.grid,
    patient_data.prescriptions,
    patient_data.structures::Dict{String, Juliana.Structure},
);

optim_grid = Juliana.get_optimisation_grid(optimisation_points, ct.grid);

In [ ]:
optim_grid

In [ ]:
main_config = Juliana.FionaStandalone.MainConfig(
    ct_path,
    output_dir,
    target_dose,
    fiona_standalone_bin_path;
    doseResolution=dose_resolution,
);

target_def = Juliana.FionaStandalone.StructureDefinition(target, 0)

optim_config = Juliana.FionaStandalone.OptimizationSettings(
    target_dose,
    0.9f0 * target_dose,
    target_def,
    Vector{Juliana.FionaStandalone.StructureConstraints}(undef, 0),
    Juliana.FionaStandalone.to_OptimizationGrid(optim_grid),
    300, # Number of iterations.
);

field = Juliana.FionaStandalone.SpotPlacementFieldDefinition(
    0,                 # target ID
    gantry_angle,      # gantry angle
    couch_angle,       # couch angle
    nozzle_extraction, # nozzle extraction
    "AUTO",
);

spot_placement_config = Juliana.FionaStandalone.SpotPlacementConfig(
    [field],
    [target_def],
);

In [ ]:
Juliana.FionaStandalone.run_spot_placement(
    fiona_jar_path,
    output_dir,
    false, # log DIj
    false, # log WED
    main_config,
    optim_config,
    spot_placement_config,
)

plan = Juliana.FionaStandalone.read_plan_file("$(output_dir)/result_plan.json")

Juliana.FionaStandalone.run_optimization(
    fiona_jar_path,
    output_dir,
    false, # log Dij
    false, # log WED
    main_config,
    optim_config,
    spot_placement_config,
    plan;
);

Juliana.FionaStandalone.run_dose_calculation(
    fiona_jar_path,
    output_dir,
    false, # log Dij
    false, # log WED
);

In [ ]:
plan = Juliana.FionaStandalone.read_plan_file("$(output_dir)/result_plan.json");
dose_fiona = Juliana.load_dose_dat_file("$(output_dir)/result_dose.dat");

# Calculate dose using Fiona

## Load machine parameters for the given field

In [ ]:
field = only(plan.fields);
spot_df = DataFrame(field.spots);

In [ ]:
ddc, sigma_dc, ps_no_preab, ps_with_preab = Juliana.load_machine_parameters(fiona_standalone_bin_path, field.nozzleExtraction);

## Define the grid

In [ ]:
@assert ct.grid.origin == zeros(3)

dose_calculation_spacing = [dose_resolution, dose_resolution, dose_resolution]
dose_calculation_origin = ct.grid.origin
# floor(): Make sure the dose calculation grid lies fully within the CT grid.
dose_calculation_size = convert.(Int64, floor.(ct.grid.spacing .* ct.grid.size ./ dose_calculation_spacing))

dose_calculation_grid = Juliana.Grid(
    dose_calculation_spacing,
    dose_calculation_origin,
    dose_calculation_size,
);

In [ ]:
dose_calculation_size

In [ ]:
points, indices = Juliana.grid_to_points_and_indices(dose_calculation_grid)

## Calculate relative stopping power

In [ ]:
hu_to_sp = Juliana.hu_to_sp_factory("$(fiona_standalone_bin_path)/huToSp.json");

In [ ]:
densities = Juliana.ScalarGrid(
    hu_to_sp.(ct.data),
    ct.grid,
);

## Calculate the WED

In [ ]:
@time wed = Juliana.calculate_wed(densities, gantry_angle, couch_angle, points);

In [ ]:
size(wed)

In [ ]:
wed_cube_coarse = Array{Float32, 3}(undef, dose_calculation_grid.size...);

## Calculate the dose

In [ ]:
dose_kernel = Juliana.DoseCalculationKernel(CUDADevice(), 32);

In [ ]:
d_dose = cu(Vector{Float32}(undef, length(wed)))
fill!(d_dose, 0)
d_wed = cu(wed)
d_field_center = cu([
    field.fieldCenter["x"],
    field.fieldCenter["y"],
    field.fieldCenter["z"],
])
d_spots_t = cu(spot_df.t)
d_spots_u = cu(spot_df.u)
d_spots_E = cu(Float32.(spot_df.energykeV))
d_preabsorbers = cu(Float32.(spot_df.numberOfAbsorbers))
d_w = cu(spot_df.weight)
d_points = cu(points')
d_ddc = cu(ddc)
d_sigma_dc = cu(sigma_dc)
d_ps_no_preab = cu(ps_no_preab)
d_ps_with_preab = cu(ps_with_preab);

In [ ]:
start = time()
event = Juliana.dose_kernel(
    d_dose,
    d_wed,
    d_field_center,
    Float32(field.gantryAngle),
    Float32(field.couchAngle),
    d_spots_t,
    d_spots_u,
    d_spots_E,
    d_preabsorbers,
    d_w,
    d_points,
    d_ddc,
    d_sigma_dc,
    d_ps_no_preab,
    d_ps_with_preab,
    Juliana.PREABSORBER_WED,
    ndrange=(length(wed)),
)
wait(event)
stop = time()
@printf "Time for dose calculation: %.2fs" stop - start

In [ ]:
dose_at_nodes = collect(d_dose)

In [ ]:
dose_at_nodes_cube = Juliana.flat_vector_to_cube(dose_calculation_grid, indices, dose_at_nodes);
dose_at_nodes = Juliana.ScalarGrid(dose_at_nodes_cube, dose_calculation_grid);

In [ ]:
points_ct, indices_ct = Juliana.grid_to_points_and_indices(ct.grid)

dose_interpolated_cube = zeros(Float32, ct.grid.size...)
for (index, p) in zip(eachcol(indices_ct), eachcol(points_ct))
    dose_interpolated_cube[index[1], index[2], index[3]] = Juliana.interpolate_linearly(dose_at_nodes, Tuple(p))
end
dose_interpolated_cube[isnan.(dose_interpolated_cube)] .= 0
dose_interpolated = Juliana.ScalarGrid(dose_interpolated_cube, ct.grid);

In [ ]:
Juliana.dicom_export_to_directory(
    ct,
    patient_data.structures,
    dicom_dir,
    "bellotti_r_tmp",
    "bellotti_r_tmp",
    Dict{String, Juliana.ScalarGrid}(
        "fiona" => dose_fiona,
        "on_dose_calc_grid" => dose_at_nodes,
        "interpolated" => dose_interpolated,
    ),
)

In [ ]:
maximum(dose_at_nodes.data)

In [ ]:
maximum(dose_fiona.data)

In [ ]:
densities.grid

In [ ]:
patient_data.prescriptions.target_doses

In [ ]:
using PyPlot

In [ ]:
fig, ax = subplots()

ax.hist(abs.(vec(dose_fiona.data) .- vec(dose_interpolated.data)), bins=100)
#ax.set_xscale("log")
ax.set_yscale("log")